<a href="https://colab.research.google.com/github/monesh1981/GEE_Projectmkb/blob/main/Assignment_Bihar_LULC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [236]:
import geemap
import ee
ee.Authenticate()
ee.Initialize(project = "ee-moneshbathre")
mapviz = geemap.Map()

Define Study Area and
Import Chattisgarh boundary from GEE Project Assets

In [241]:
study_area = ee.FeatureCollection('projects/ee-moneshbathre/assets/bihar_bdy')
mapviz.addLayer(study_area,{},'Chattisgarh')
mapviz.centerObject(study_area)


#adding legend and visualisation parameters for lulc as per ESA Worldcover

In [242]:
value_visual = {'min':10,'max':100,'palette':['#006400','#ffbb22','#ffff4c','#f096ff','#fa0000','#b4b4b4','#f0f0f0','#0064c8','#0096a0','	#00cf75','#fae6a0']}
Legend = {'Tree Cover':'#006400',
          'Shrubland':'#ffbb22',
          'Grassland':'#ffff4c',
          'Cropland':'#f096ff',
          'Built-up':'#fa0000',
          'Bare / sparse vegetation':'#b4b4b4',
          'Snow and ice':'#f0f0f0',
          'Permanent water bodies':'#0064c8',
          'Herbaceous wetland':'#0096a0',
          'Mangroves':'#00cf75',
          'Moss and lichen':'#fae6a0'}

Clip ESA World Cover Data limited to Chattisgarh studay area

In [243]:
cg_esa = ee.ImageCollection('ESA/WorldCover/v200').mean().clip(study_area).select('Map')
mapviz.addLayer(cg_esa,value_visual,'Bihar ESA Worldcover')


In [244]:
mapviz.add_legend(title="My Legend", legend_dict=Legend, **value_visual)

**Clip Landsat 8 data for study area and dates/cloudcover**


In [245]:
landsat8_data = ee.ImageCollection("LANDSAT/LC08/C02/T1").filterDate('2024-01-01', '2024-12-31').filterMetadata('CLOUD_COVER','less_than',5).mean().clip(study_area)
mapviz.addLayer(landsat8_data,{},'Landsat8_2024')
mapviz

Map(bottom=7366.0, center=[25.903891043806432, 85.80700650480884], controls=(WidgetControl(options=['position'…

**Adding Training Points**

In [248]:
TP = cg_esa.sample(**{
    'region':study_area.geometry(),
    'scale':20,
    'numPixels':4000,
    'seed':1,
    'geometries':True
    })
mapviz.addLayer(TP,{},'Training Points')
mapviz



Map(bottom=56132.0, center=[25.740529092773226, 86.36873349177245], controls=(WidgetControl(options=['position…

In [249]:
bands = ['B1','B2','B3','B4','B5','B6']
label = 'Map'
training = landsat8_data.select(bands).sampleRegions(**{'collection':TP,
                                       'properties':[label],
                                       'scale':30})
trained = ee.Classifier.smileKNN().train(training,label,bands)
result = landsat8_data.select(bands).classify(trained)

In [251]:
mapviz.addLayer(result,value_visual,'Supervised_classification')
mapviz

Map(bottom=3630.0, center=[31.87755764334002, 104.8077625964531], controls=(WidgetControl(options=['position',…